In [ ]:
# Suppress Warning messages
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
PROBLEM_NAME = 'KColor'
ALGORITHM_NAME = 'SA'

In [ ]:
PROBLEM_SIZE = 100
CONNECTIONS_PER_NODE = 4

In [ ]:
out_dir = f'metrics'
os.makedirs(out_dir, exist_ok=True)

In [ ]:
all_df = pd.read_csv(f'metrics/{ALGORITHM_NAME.lower()}.csv')

In [ ]:
all_df.columns

In [ ]:
print(all_df['decay_type'].unique())
print(all_df['Temperature'].unique())

In [ ]:
all_df = all_df[all_df['Temperature'].isin([1, 100, 10000])]

In [ ]:
all_df['X'] = all_df.apply(lambda row: f"Temp={row['Temperature']}, DecayType={row['decay_type']}", axis=1)

In [ ]:
all_df['X'].unique()

In [ ]:
def get_color(X):
    if 'DecayType=ExpDecay' in X:
        return 'red'
    elif 'DecayType=GeomDecay' in X:
        return 'blue'
    elif 'DecayType=ArithDecay' in X:
        return 'green'
    else:
        raise Exception(f'Unknown X: {X}')
def get_style(X):
    # if X[-6:] == 'Temp=1':
    #     return 'dotted'
    # elif X[-8:] == 'Temp=100':
    #     return '--'
    # elif X[-10:] == 'Temp=10000':
    #     return '-'
    if 'Temp=1,' in X:
        return 'dotted'
    elif 'Temp=100,' in X:
        return '--'
    elif 'Temp=10000,' in X:
        return '-'

In [ ]:
list(range(1, 100, 10))

In [ ]:
metrics = {}

for group in sorted(all_df['X'].unique()):
    df = all_df[all_df['X'] == group]
    assert(len(df['group_timestamp'].unique()) == 1)
    last_iter = df['Iteration'].max()
    metrics[group] = {
        'iter': [],
        'fitness_mean': [],
        'fitness_std': [],
    }
    for iter_i in range(1, last_iter+1, 10):
        iter_df = df[df['Iteration'] == iter_i]
        metrics[group]['iter'].append(iter_i)
        metrics[group]['fitness_mean'].append(iter_df['Fitness'].mean())
        metrics[group]['fitness_std'].append(iter_df['Fitness'].std())

In [ ]:
fontsize = 16
plt.figure(figsize=(12, 8))
last_iter = all_df['Iteration'].max()
# plt.hlines(30, 0, last_iter, color='red', linestyle='dashed', alpha=1.0, label='Local Optima (Fitness=30)')
# plt.hlines(47, 0, last_iter, color='black', linestyle='dashed', alpha=1.0, label='Global Optima (Fitness=47)')
for group in metrics:
    linestyle = get_style(group)
    color = get_color(group)
    # if color != 'red':
    #     continue
    plt.plot(metrics[group]['iter'], metrics[group]['fitness_mean'], label=group, color=color, linestyle=linestyle)
    mean = np.array(metrics[group]['fitness_mean'])
    std = np.array(metrics[group]['fitness_std']) / 4
    lower = mean - std
    upper = mean + std 
    plt.fill_between(metrics[group]['iter'], lower, upper, alpha=0.2, color=color)
plt.title(f'{PROBLEM_NAME} - {ALGORITHM_NAME} - Fitness vs Iteration', fontsize=fontsize)
plt.xlabel('Iteration', fontsize=fontsize)
plt.ylabel('Fitness', fontsize=fontsize)
# plt.xlim(0, 4000)
plt.legend()
plt.savefig(f'{out_dir}/{PROBLEM_NAME}_{ALGORITHM_NAME.lower()}_fitness_vs_iteration.png')